In [1]:
from hops import hdfs
import datetime
from pyspark.sql import DataFrame, Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, col
import sys

#num_rows=100000
num_rows=1000000
#num_rows=10000000

Starting Spark application


The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [2]:
fg1_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f1", StringType(), True)    
])

fg1=spark.read.csv(hdfs.project_path() + "Resources/" + str(num_rows) + "-20-1-out.csv", header=True, schema=fg1_schema)
fg1=fg1.sort(col("id"),col("ts"))
fg1.show(4)

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [3]:
fg1.count()

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [4]:
# fg2_schema = StructType([
#   StructField("id", IntegerType(), True),
#   StructField("ts", IntegerType(), True),
#   StructField("f2", StringType(), True)
# ])
# , schema=fg2_schema
fg2=spark.read.csv(hdfs.project_path() + "Resources/" + str(num_rows) + "-20-2-out.csv", header=True)
fg2=fg2.sort(col("id"),col("ts"))
fg2.show(4)

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [5]:
fg2.count()

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [6]:
maprdd = fg2.rdd.groupBy(lambda x:x[0]).map(lambda x:(x[0],{y[1] for y in x[1]}))
result_dict = dict(maprdd.collect()) 

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [7]:
map_broadcast = sc.broadcast(result_dict)

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [8]:
def take_closest(id, ts):
    return min(map_broadcast.value[id],key=lambda x:((ts-x) if ts >= x else sys.maxsize))
columns = ["id2","join_ts","ts2"]

my_rdd = fg1.rdd.map(lambda x: (x[0],x[1],take_closest(x[0],x[1])))
filtered_rdd = my_rdd.filter(lambda x: x[1] >= x[2])
join2=spark.createDataFrame(filtered_rdd,columns).sort(col("id2"),col("join_ts"))
join2.show()

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [9]:
joined = fg1.join(join2,(fg1.id==join2.id2) & (fg1.ts==join2.join_ts),how="inner")
drop_cols = ['id2', 'join_ts']
inter = joined.drop(*drop_cols)
inter.show(15)

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [10]:
inter = inter.alias('inter')
fg2 = fg2.alias('fg2')
final = inter.join(fg2,(inter.id==fg2.id) & (inter.ts2==fg2.ts),how="inner")
#.select('inter.id', 'inter.ts', 'inter.f1', 'fg2.f2')
final.show(15)

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl

In [11]:
final.count()

The code failed because of a fatal error:
	Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 
2021-02-10 22:07:18,917 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-10 22:07:19,031 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2021-02-10 22:07:19,210 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.5:8032
2021-02-10 22:07:20,151 INFO  Client: Requesting a new application from cluster with 7 NodeManagers
2021-02-10 22:07:20,264 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2021-02-10 22:07:20,274 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2021-02-10 22:07:20,291 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the cl